In [2]:
#coding=utf-8
import numpy as np
label_names = ['food','price','service','ambience','anecdotes/miscellaneous']
label_names_simple = ['food','price','service','ambience','anecdotes']
polarity_names = ['positive','conflict','negative','neutral']


In [3]:
from document import Document
document = Document()
document.set_xml_path("ABSA2014/Train/Restaurants_Train.xml")
document.set_pkl_path("train.pkl")
train = document.get_data()
document.set_xml_path("ABSA2014/Gold/Restaurants_Gold.xml")
document.set_pkl_path("gold.pkl")
test = document.get_data()
for num, one in enumerate(train[:20]):
    print "#{}:{}".format(num, one)

#0:{'text': 'But the staff was so horrible to us.', 'aspectCategories': [{'category': 'service', 'polarity': 'negative'}]}
#1:{'text': "To be completely fair, the only redeeming factor was the food, which was above average, but couldn't make up for all the other deficiencies of Teodora.", 'aspectCategories': [{'category': 'food', 'polarity': 'positive'}, {'category': 'anecdotes/miscellaneous', 'polarity': 'negative'}]}
#2:{'text': "The food is uniformly exceptional, with a very capable kitchen which will proudly whip up whatever you feel like eating, whether it's on the menu or not.", 'aspectCategories': [{'category': 'food', 'polarity': 'positive'}]}
#3:{'text': 'Where Gabriela personaly greets you and recommends you what to eat.', 'aspectCategories': [{'category': 'service', 'polarity': 'positive'}]}
#4:{'text': "For those that go once and don't enjoy it, all I can say is that they just don't get it.", 'aspectCategories': [{'category': 'anecdotes/miscellaneous', 'polarity': 'positive

In [4]:
def encode_label(labels):
    return [[label_names.index(label) for label in line] for line in labels]

In [5]:
def get_task1_data(data):
    raw_x = []
    vocabsLabel = []
    for sentence in data:
        labels = sentence['aspectCategories']
        vocabs = sentence['text']
        raw_x.append(vocabs)
        labelsForData = []
        for label in labels:
            labelsForData.append(label['category'])
        vocabsLabel.append(labelsForData)
    from sklearn.preprocessing import MultiLabelBinarizer
    y_mul = MultiLabelBinarizer().fit_transform(encode_label(vocabsLabel))
    return y_mul, raw_x

In [6]:
y_train, raw_x_train = get_task1_data(train)
y_test, raw_x_test = get_task1_data(test)
import preprocess
# vectorize the data
x_train = preprocess.preprocessing(raw_x_train)
x_test = preprocess.preprocessing(raw_x_test)
print len(x_train)
print len(y_train)
for num, (x_, y_) in enumerate(zip(x_train[:20], y_train[:20])):
    print "#{}: x:{}, y:{}".format(num, x_, y_)

/home/cer/anaconda2/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


3044
3044
#0: x:[u'staff', u'horrible', u'us'], y:[0 0 1 0 0]
#1: x:[u'completely', u'fair', u'redeeming', u'factor', u'food', u'average', u'could', u"n't", u'make', u'deficiencies', u'teodora'], y:[1 0 0 0 1]
#2: x:[u'food', u'uniformly', u'exceptional', u'capable', u'kitchen', u'proudly', u'whip', u'whatever', u'feel', u'like', u'eating', u'whether', u"'s", u'menu'], y:[1 0 0 0 0]
#3: x:[u'gabriela', u'personaly', u'greets', u'recommends', u'eat'], y:[0 0 1 0 0]
#4: x:[u'go', u"n't", u'enjoy', u'say', u"n't", u'get'], y:[0 0 0 0 1]
#5: x:[u'food', u'outstanding', u'little', u"'perks", u"'", u'great'], y:[1 0 1 0 0]
#6: x:[u'overpriced', u'tasty'], y:[1 1 0 0 0]
#7: x:[u'agreed', u'favorite', u'orrechiete', u'sausage', u'chicken', u'usually', u'waiters', u'kind', u'enough', u'split', u'dish', u'half', u'get', u'sample', u'meats'], y:[1 0 1 0 0]
#8: x:[u'bagels', u'outstanding', u'taste', u'terrific', u'texture', u'chewy', u'yet', u'gummy'], y:[1 0 0 0 0]
#9: x:[u'nevertheless', u'food

In [7]:
# generate the dictionary
dictionary = preprocess.get_dict(x_train)
for key in dictionary.keys()[:5]:
    print "key:{}, value:{}".format(key, dictionary[key])
print "words num: {}".format(len(dictionary.keys()))

key:2160, value:rasamalai
key:140, value:secondly
key:860, value:legacies
key:387, value:limited
key:2213, value:ciao
words num: 4469


In [8]:
# transform to tf-idf
x_train_ti_cor = preprocess.calc_tfidf(x_train, dictionary)
x_test_ti_cor = preprocess.calc_tfidf(x_test, dictionary)
from gensim import matutils
x_train_ti_csc = matutils.corpus2csc(x_train_ti_cor).transpose()
x_test_ti_csc = matutils.corpus2csc(x_test_ti_cor).transpose()
print x_train_ti_csc.shape
print y_train.shape

(3044, 4469)
(3044, 5)


In [9]:
def evaluate_cross_validation(clf,X,y,K=10):
    from sklearn.cross_validation import cross_val_score, KFold
    from scipy.stats import sem
    cv = KFold(len(y),K,shuffle=True,random_state=1)
    scores = cross_val_score(clf,X,y,cv=cv)
    print scores
    print("Mean score: {0:.3f} (+/-{1:.3f})").format(np.mean(scores),sem(scores))


In [13]:
def train_and_evaluate(clf,X_train,X_test,y_train,y_test):
    clf.fit(X_train,y_train)
    print "Accuracy on training set:"
    print clf.score(X_train,y_train)
    print "Acuracy on testing set:"
    print clf.score(X_test,y_test)


In [14]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(SVC(kernel='linear', C=1.))
print "The Result of Task 1:"
train_and_evaluate(clf, x_train_ti_csc, 
                   x_test_ti_csc, y_train, y_test)

The Result of Task 1:
Accuracy on training set:
0.853810775296
Acuracy on testing set:
0.816857440166


In [15]:
from gensim import models
lda = models.LdaModel(x_train_ti_cor, id2word=dictionary, num_topics=5)

In [16]:
lda.print_topics()

[(0,
  u'0.005*"like" + 0.004*"never" + 0.004*"\'s" + 0.004*"get" + 0.004*"food" + 0.004*"great" + 0.004*"one" + 0.003*"went" + 0.003*"restaurant" + 0.003*"\'ve"'),
 (1,
  u'0.008*"food" + 0.008*"go" + 0.007*"good" + 0.006*"place" + 0.006*"delicious" + 0.006*"would" + 0.005*"great" + 0.005*"highly" + 0.005*"wonderful" + 0.005*"back"'),
 (2,
  u'0.012*"service" + 0.011*"food" + 0.009*"great" + 0.008*"good" + 0.005*"place" + 0.005*"n\'t" + 0.004*"prices" + 0.004*"also" + 0.004*"staff" + 0.004*"fantastic"'),
 (3,
  u'0.007*"n\'t" + 0.006*"place" + 0.005*"ca" + 0.004*"wait" + 0.004*"restaurant" + 0.004*"back" + 0.004*"food" + 0.003*"good" + 0.003*"great" + 0.003*"\'s"'),
 (4,
  u'0.004*"food" + 0.004*"good" + 0.003*"service" + 0.003*"bagel" + 0.003*"n\'t" + 0.003*"waiter" + 0.003*"wrong" + 0.003*"average" + 0.003*"nice" + 0.003*"coming"')]

In [26]:
for num, one in enumerate(lda[x_ti_cor]):
    print "#{} document:{}".format(num, one)

#0 document:[(0, 0.0370293719516578), (1, 0.037011284520995148), (2, 0.037024607626060892), (3, 0.037024589349306899), (4, 0.037015442905230199), (5, 0.037017370023492674), (6, 0.03701834524362204), (7, 0.037017179148502284), (8, 0.03702646877305605), (9, 0.666815340458076)]
#1 document:[(0, 0.027405671982324839), (1, 0.027406233939639844), (2, 0.02741741497354171), (3, 0.027406793164966182), (4, 0.027418978804297653), (5, 0.027437158573638383), (6, 0.75328437083677147), (7, 0.027407866193859723), (8, 0.027408906944616746), (9, 0.027406604586343388)]
#2 document:[(0, 0.024317698851345645), (1, 0.024318231172297856), (2, 0.69125319550581132), (3, 0.024322802962370552), (4, 0.11419430651557332), (5, 0.02431714774315125), (6, 0.024318989848839068), (7, 0.024320273539853401), (8, 0.024320360160189684), (9, 0.024316993700567929)]
#3 document:[(0, 0.050009120866262989), (1, 0.050018169422030107), (2, 0.050013175233879673), (3, 0.05000713509108741), (4, 0.050010088580530784), (5, 0.0500143424